In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
data=pd.read_csv('../input/international-football-results-from-1872-to-2017/results.csv')
data.head()

In [ ]:
data.columns

In [ ]:
data["Year"]=pd.to_datetime(data['date']).dt.year
data["Month"]=pd.to_datetime(data['date']).dt.month
data=data.drop(columns=['date'], axis=1)
data.head()

In [ ]:
data['total_score']=data['home_score']+data['away_score']
data.head()

In [ ]:
con=[(data['home_score']==data['away_score']),(data['home_score']>data['away_score']),(data['home_score']<data['away_score'])]
val=[0,1,2]
val2=['no_win',data['home_team'],data['away_team']]
data['result']=np.select(con,val)
data['who_win']=np.select(con,val2)
data.head()

In [ ]:
# Who is the best team of all time
# 1- teams with the most goals
bestteam=data.groupby('who_win').sum()
bestteam=bestteam[['home_score','away_score','total_score']].sort_values(by=['total_score'],ascending=False)
bestteam=bestteam.drop(['no_win'])
bestteam.head(10)

In [ ]:
#2- most winning teams
bestteam2=data.groupby('who_win').count()
bestteam2=bestteam2[['result']].sort_values(by=['result'],ascending=False)
bestteam2=bestteam2.drop(['no_win'])

bestteam2.head(10)

In [ ]:
#3- merge most winning teams and gols
bestteam3=pd.merge(bestteam,bestteam2, how='inner', on='who_win')
bestteam3['teams']=bestteam3.index
bestteam3.head()

In [ ]:
#4-the number of matches they played
match=data.groupby('home_team').count()+data.groupby('away_team').count()
match=match[['city']].sort_values(by=['city'],ascending=False)
match=match.rename(columns = {'city':'play_count'})
match['teams']=match.index
match.head()

In [ ]:
#5-merge the number of matches they played and winner
bestteam4=pd.merge(match,bestteam3,how='inner', on='teams')
bestteam4['win_per_game']=bestteam4['result']/bestteam4['play_count']
bestteam4['goal_per_game']=bestteam4['total_score']/bestteam4['play_count']
bestteam4=bestteam4[['teams', 'play_count', 'total_score', 'result', 'win_per_game', 'goal_per_game']]
bestteam4.head()

In [ ]:
bestteam4=bestteam4.sort_values(by=['win_per_game'],ascending=False)
bestteam4.head(10)

In [ ]:
bestteam4.plot(kind='scatter',x='play_count',y='win_per_game',color='r',alpha=0.5)
plt.xlabel('play_count')
plt.ylabel('win_per_game')
plt.title('play_count and win_per_game Scatter Plot')
plt.show

In [ ]:
bestteam5=bestteam4[(bestteam4['win_per_game']>0,4) and (bestteam4['play_count']>800)]
bestteam5.head()

In [ ]:
bestteam5.plot(kind='scatter',x='play_count',y='win_per_game',color='r',alpha=0.5,label='teams')
plt.xlabel('play_count')
plt.ylabel('win_per_game')
plt.title('play_count and win_per_game Scatter Plot')
plt.show

In [ ]:
# Create the figure and axes objects
fig, ax = plt.subplots(1, figsize=(10, 6))
fig.suptitle('play_count and win_per_game Scatter Plot')

# Plot the scatter points
x=bestteam5['play_count']
y=bestteam5['win_per_game']
labels=bestteam5['teams']
ax.scatter(x,y,
           color="blue",  # Color of the dots
           s=100,         # Size of the dots
           alpha=0.5,     # Alpha of the dots
           linewidths=1)  # Size of edge around the dots

# Add the participant names as text labels for each point
for x_pos, y_pos, label in zip(x, y, labels):
    ax.annotate(label,             # The label for this point
                xy=(x_pos, y_pos), # Position of the corresponding point
                xytext=(7, 0),     # Offset text by 7 points to the right
                textcoords='offset points', # tell it to use offset points
                ha='left',         # Horizontally aligned to the left
                va='center')       # Vertical alignment is centered

plt.show()

In [ ]:
# Which teams dominated different eras of football

data.head()

In [ ]:
teams=data[['Year','who_win']]
teams.head()

In [ ]:
def re_find(who_win):
    if "no_win" not in who_win.lower():
        return True
    return False

teams2=teams[teams['who_win'].apply(re_find)]

teams2.head()

In [ ]:
list1=[]
list2=[]
list3=[]
b=teams2['Year'].count()
for i in range(0,b):
    list1.append(teams2['Year'].iloc[i])
    list2.append(teams2['who_win'].iloc[i])
    a=str(teams2['Year'].iloc[i])+teams2['who_win'].iloc[i]
    list3.append(a)
    
list_label = ["Year","who_win","Year_who_win"] 
list_col = [list1,list2,list3]       
zipped = zip(list_label,list_col) 
data_dict = dict(zipped)  

teams3 = pd.DataFrame(data_dict)
teams3.head()    

In [ ]:
teams4=teams3.groupby('Year_who_win').count()
teams4=teams4[['who_win']].sort_values(by=['who_win'], ascending=False)
teams4.head()

In [ ]:
#What trends have there been in international football throughout the ages - 
#home advantage, total goals scored, distribution of teams' strength et

data.head()

In [ ]:
con=[(data['home_team']==data['who_win']),(data['away_team']==data['who_win'])]
val=['home_win','away_win']
val2=[1,1]
data['home_away_win']=np.select(con,val)
data['home_away_win2']=np.select(con,val2)
data.head()

In [ ]:
#home advantage?
data1=data.groupby('home_away_win').count()
data1[['who_win']]

In [ ]:
my_colors = ['lightblue','lightsteelblue','silver']
my_explode = (0, 0.1, 0)
plt.pie(data1['who_win'], labels=data1.index, autopct='%1.1f%%', startangle=15, shadow = True, colors=my_colors, explode=my_explode)
plt.title('Which teams are advantageous')
plt.axis('equal')
plt.show()

In [ ]:
#how has the number of countries changed 
data_c=data[['Year','country']]
data_c = data_c.drop_duplicates()
data_c1=data_c.groupby('Year').count()

plt.plot(data_c1.index, data_c1['country'], color='red', marker='o')
plt.title('Year and number of country ', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ylabel('country', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
#Which countries host the most matches where they themselves are not participating in
data_c = data.drop(data[data.country == data.home_team].index)
list1=[]
list2=[]
b=len(data_c.index)
for i in range(0,b):
    list1.append(data_c['country'].iloc[i])
    list2.append(1)

list_label = ["Country","Num"] 
list_col = [list1,list2]       
zipped = zip(list_label,list_col) 
data_dict = dict(zipped) 

data_c3 = pd.DataFrame(data_dict) 
data_c4=data_c3.groupby('Country').count()
data_c5=data_c4[['Num']]
data_c6=data_c5.sort_values(by=['Num'], ascending=False)
data_c6.head()